<a href="https://colab.research.google.com/github/cdelia/ai_colabs/blob/main/gradioOpenAISystemPrompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a simple gradio ChatBot Using OpenAI.
It features streaming responses.


Let's install the requirements.

In [4]:
pip install --q gradio openai

Now let's enter an OpenAI key.
You can create one here: [page](https://platform.openai.com/account/api-keys)


---



In [5]:
import getpass as getpass

OPENAI_API_KEY = getpass.getpass("Enter OpenAI API Key: ")

Enter OpenAI API Key: ··········


In [ ]:
import openai
import gradio as gr

openai.api_key = OPENAI_API_KEY

def predict(message, history, model, system_prompt, temperature):
    history_openai_format = []
    history_openai_format.append({"role": "system", "content": system_prompt})
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human })
        history_openai_format.append({"role": "assistant", "content":assistant})
    history_openai_format.append({"role": "user", "content": message})

    response = openai.ChatCompletion.create(
        model=model,
        messages= history_openai_format,
        temperature=temperature,
        stream=True
    )

    partial_message = ""
    for chunk in response:
        if len(chunk['choices'][0]['delta']) != 0:
            partial_message = partial_message + chunk['choices'][0]['delta']['content']
            yield partial_message

demo = gr.ChatInterface(predict,
                        additional_inputs=[
                            gr.Radio(["gpt-3.5-turbo", "gpt-3.5-turbo-16k", "gpt-4", "gpt-4-32k"], value="gpt-3.5-turbo", label="Model"),
                            gr.Textbox("You are helpful AI.", label="System Prompt"),
                            gr.Slider(0.0, 1.0, label="Temperature", step=0.01, value=0.9)
                        ]
                       )

demo.queue().launch(debug=True)
